In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
data = spark.read.json('s3://amazon-reviews-ml/json/train/dataset_en_train.json')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1685156209583_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package("boto3==1.19.2")
sc.install_pypi_package("pandas==1.0.5")
sc.install_pypi_package("scipy==1.4.1")
sc.install_pypi_package("matplotlib==3.2.1")
sc.install_pypi_package("seaborn==0.10.1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#sc.install_pypi_package("torch")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#sc.install_pypi_package("transformers==4.2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.install_pypi_package("spark-nlp")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
sc.install_pypi_package("textblob")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print('Total Columns: %d' % len(data.dtypes))
print('Total Rows: %d' % data.count())
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Columns: 8
Total Rows: 200000
root
 |-- language: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- review_title: string (nullable = true)
 |-- reviewer_id: string (nullable = true)
 |-- stars: string (nullable = true)

In [9]:
data[['product_category', 'review_body', 'stars']].show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-----+
|   product_category|         review_body|stars|
+-------------------+--------------------+-----+
|          furniture|Arrived broken. M...|    1|
|   home_improvement|the cabinet dot w...|    1|
|               home|I received my fir...|    1|
|           wireless|This product is a...|    1|
|                 pc|went through 3 in...|    1|
|industrial_supplies|Poor quality. The...|    1|
|               home|Ordered 2 they sh...|    1|
|            kitchen|Followed directio...|    1|
|            apparel|There is a terrib...|    1|
|         automotive|Unless you have t...|    1|
|            apparel|I am disappointed...|    1|
|             camera|It doesn’t work a...|    1|
|           wireless|was not pleased a...|    1|
|    lawn_and_garden|Followed instruct...|    1|
|           wireless|Absolutely horrib...|    1|
|           wireless|Does not play wit...|    1|
|             camera|Doesn’t work and ...|    1|
|              watch

In [10]:
drop_list = ['language', 'review_id']
data = data.select([column for column in data.columns if column not in drop_list])
data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+--------------------+--------------------+-------------------+-----+
|product_category|        product_id|         review_body|        review_title|        reviewer_id|stars|
+----------------+------------------+--------------------+--------------------+-------------------+-----+
|       furniture|product_en_0740675|Arrived broken. M...|I'll spend twice ...|reviewer_en_0342986|    1|
|home_improvement|product_en_0440378|the cabinet dot w...|        Not use able|reviewer_en_0133349|    1|
|            home|product_en_0399702|I received my fir...|The product is junk.|reviewer_en_0152034|    1|
|        wireless|product_en_0444063|This product is a...|Fucking waste of ...|reviewer_en_0656967|    1|
|              pc|product_en_0139353|went through 3 in...|              bubble|reviewer_en_0757638|    1|
+----------------+------------------+--------------------+--------------------+-------------------+-----+
only showing top 5 rows

In [11]:
# top 10 product category
from pyspark.sql.functions import col
data.groupBy("product_category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|product_category|count|
+----------------+-----+
|            home|17679|
|         apparel|15951|
|        wireless|15717|
|           other|13418|
|          beauty|12091|
|       drugstore|11730|
|         kitchen|10382|
|             toy| 8745|
|          sports| 8277|
|      automotive| 7506|
+----------------+-----+
only showing top 10 rows

In [12]:
# top 10 repeated products
data.groupBy("product_id") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|        product_id|count|
+------------------+-----+
|product_en_0184780|    6|
|product_en_0209329|    5|
|product_en_0061393|    5|
|product_en_0584134|    5|
|product_en_0977951|    5|
|product_en_0311784|    5|
|product_en_0360338|    5|
|product_en_0538497|    5|
|product_en_0420950|    5|
|product_en_0491284|    5|
+------------------+-----+
only showing top 10 rows

In [13]:
data.groupBy("reviewer_id") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|        reviewer_id|count|
+-------------------+-----+
|reviewer_en_0857640|    5|
|reviewer_en_0953455|    4|
|reviewer_en_0648912|    4|
|reviewer_en_0078203|    4|
|reviewer_en_0012654|    4|
|reviewer_en_0358974|    4|
|reviewer_en_0371968|    4|
|reviewer_en_0957630|    4|
|reviewer_en_0920760|    4|
|reviewer_en_0033216|    4|
+-------------------+-----+
only showing top 10 rows

### Model Pipeline

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Use TF-IDF

In [86]:
# from pyspark.ml.feature import HashingTF, IDF
# hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
# idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

# titleTF = HashingTF(inputCol="title_words", outputCol="rawFeatures1", numFeatures=10000)
# title_idf = IDF(inputCol="rawFeatures1", outputCol="title_features", minDocFreq=5)

# pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, tokenizer_title, titleTF, title_idf, y_stringidx])
# pipelineFit = pipeline.fit(data)
# dataset = pipelineFit.transform(data)
# (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
# lr = LogisticRegression()
# lrModel = lr.fit(trainingData)
# predictions = lrModel.transform(testData)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
# evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
# evaluator.evaluate(predictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
# from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# # Create ParamGrid for Cross Validation
# paramGrid = (ParamGridBuilder()
#              .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
#              .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
# #            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
# #            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
#              .build())
# # Create 5-fold CrossValidator
# cv = CrossValidator(estimator=lr, \
#                     estimatorParamMaps=paramGrid, \
#                     evaluator=evaluator, \
#                     numFolds=5)
# cvModel = cv.fit(trainingData)

# predictions = cvModel.transform(testData)
# # Evaluate best model
# evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
# evaluator.evaluate(predictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
# ## random forest
# from pyspark.ml.classification import RandomForestClassifier
# rf = RandomForestClassifier(labelCol="label", \
#                             featuresCol="features", \
#                             numTrees = 100, \
#                             maxDepth = 4, \
#                             maxBins = 32)
# # Train model with Training Data
# rfModel = rf.fit(trainingData)
# predictions = rfModel.transform(testData)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Build model with sentiment analysis

In [17]:
from pyspark.sql.types import FloatType 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from textblob import TextBlob

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
from pyspark.sql.functions import udf

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql.functions import concat_ws

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from textblob import TextBlob
# Start Spark Session
spark = SparkSession.builder \
    .appName("TextBlob Sentiment Analysis") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
def sentiment_score(chat):
    return TextBlob(chat).sentiment.polarity

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Define and register UDF for sentiment polarity
sentiment_polarity = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register("sentiment_polarity", sentiment_polarity)

# Define and register UDF for sentiment subjectivity
sentiment_subjectivity = udf(lambda x: TextBlob(x).sentiment[1])
spark.udf.register("sentiment_subjectivity", sentiment_subjectivity)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7ff12a835830>

In [24]:
data = data.withColumn("Text", concat_ws('', 'review_title', 'review_body'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
texts = data.withColumn('sentiment_polarity', sentiment_polarity('Text').cast('double'))
texts = texts.withColumn('sentiment_subjectivity', sentiment_subjectivity('Text').cast('double'))

texts.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+--------------------+--------------------+-------------------+-----+--------------------+------------------+----------------------+
|product_category|        product_id|         review_body|        review_title|        reviewer_id|stars|                Text|sentiment_polarity|sentiment_subjectivity|
+----------------+------------------+--------------------+--------------------+-------------------+-----+--------------------+------------------+----------------------+
|       furniture|product_en_0740675|Arrived broken. M...|I'll spend twice ...|reviewer_en_0342986|    1|I'll spend twice ...|           -0.1625|   0.38958333333333334|
|home_improvement|product_en_0440378|the cabinet dot w...|        Not use able|reviewer_en_0133349|    1|Not use ablethe c...|               0.0|                   0.0|
+----------------+------------------+--------------------+--------------------+-------------------+-----+--------------------+------------------+----------

In [26]:
data = texts.select('Text','product_category', 'sentiment_polarity','sentiment_subjectivity', 'stars')
data = data.withColumn('pos_sentiment', col('sentiment_polarity') + 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
tokenizer_title = RegexTokenizer(inputCol="Text", outputCol="body", pattern = "\\W")
countVectors_title = CountVectorizer(inputCol="body", outputCol="feature", vocabSize=10000, minDF=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
cat_stringIdx = StringIndexer(inputCol = "product_category", outputCol = "pro_cat")
y_stringidx = StringIndexer(inputCol = "stars", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer_title, countVectors_title, cat_stringIdx, y_stringidx])
pipelineFit = pipeline.fit(data)
data = pipelineFit.transform(data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
data.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+-------------------+----------------------+-----+------------------+--------------------+--------------------+-------+-----+
|                Text|product_category| sentiment_polarity|sentiment_subjectivity|stars|     pos_sentiment|                body|             feature|pro_cat|label|
+--------------------+----------------+-------------------+----------------------+-----+------------------+--------------------+--------------------+-------+-----+
|I'll spend twice ...|       furniture|            -0.1625|   0.38958333333333334|    1|            0.8375|[i, ll, spend, tw...|(10000,[0,1,2,3,4...|   21.0|  0.0|
|Not use ablethe c...|home_improvement|                0.0|                   0.0|    1|               1.0|[not, use, ableth...|(10000,[8,38,41,5...|   11.0|  0.0|
|The product is ju...|            home|               0.25|   0.29166666666666663|    1|              1.25|[the, product, is...|(10000,[0,1,2,3,6...|    0.0|  0.0|
|Fucking waste o

In [30]:
features = ["feature","pos_sentiment", "sentiment_subjectivity", "pro_cat", "label"]
assembler = VectorAssembler(inputCols = features, outputCol = 'features')
transformed_data = assembler.transform(data.na.drop())
train, test = transformed_data.randomSplit([0.7, 0.3], seed = 1203)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# naive bayes
nb = NaiveBayes(modelType="multinomial",labelCol="label", featuresCol="features")
nbparamGrid = (ParamGridBuilder()
               .addGrid(nb.smoothing, np.arange(1, 40, 1))
               .build())
nbevaluator = MulticlassClassificationEvaluator(labelCol="label", 
                                                predictionCol="prediction", 
                                                metricName="accuracy")
nbcv = CrossValidator(estimator = nb,
                      estimatorParamMaps = nbparamGrid,
                      evaluator = nbevaluator,
                      numFolds = 5)
nbmodel = nbcv.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
nbpredictions = nbmodel.transform(test)
print('Accuracy:', nbevaluator.evaluate(nbpredictions))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6235739542331042

In [34]:
# logistic
lr = LogisticRegression(standardization = True)
grid = ParamGridBuilder().addGrid(lr.regParam, np.arange(0,1,.1)) \
.addGrid(lr.elasticNetParam, [0, 0.5, 1]).build()
evaluator = MulticlassClassificationEvaluator()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator,
    parallelism=320, numFolds = 5)
cvModel = cv.fit(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
evaluationSummary = cvModel.bestModel.transform(test)
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(evaluationSummary)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8536423417728175

In [36]:
from pyspark.sql.functions import *
prediction = evaluationSummary.select('label', 'prediction')
prediction = prediction.withColumn('true', (prediction.prediction == prediction.label).cast("integer")).persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
prediction.select(mean('true')).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(avg(true)=0.853365785349915)]

In [38]:
test1 = test.filter("label == 0.0").persist()
test2 = test.filter("label == 1.0").persist()
test3 = test.filter("label == 2.0").persist()
test4 = test.filter("label == 3.0").persist()
test5 = test.filter("label == 4.0").persist()
test_datas = [test1,test2,test3,test4,test5]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
for testdata in test_datas:
    evaluationSummary = cvModel.bestModel.transform(testdata)
    prediction = evaluationSummary.select('label', 'prediction')
    prediction = prediction.withColumn('true', (prediction.prediction == prediction.label).cast("integer")).persist()
    prediction.select(mean('true')).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(avg(true)=0.9187290969899665)]
[Row(avg(true)=0.8597402597402597)]
[Row(avg(true)=0.8075058508859914)]
[Row(avg(true)=0.7915080284721072)]
[Row(avg(true)=0.8897467828974678)]

In [40]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(numTrees=100, maxDepth=30, labelCol="label", seed=1203,
    leafCol="leafId")
model = rf.fit(train)
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.8577569936298515
Test Error = 0.14224300637014853

In [41]:
rf = RandomForestClassifier(numTrees=200, maxDepth=30, labelCol="label", seed=1203,
    leafCol="leafId")
model = rf.fit(train)
predictions = model.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.8536572515403611
Test Error = 0.1463427484596389